In [73]:
# !pip install spacy

from dotenv import load_dotenv
import ast

load_dotenv()

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/andriimyronenko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [28]:
import pandas as pd

test_tech_df = pd.read_csv('./data/test_df_tech_only_combined.csv')

test_tech_df['knowledge'] = test_tech_df['knowledge'].apply(ast.literal_eval)
test_tech_df['skills'] = test_tech_df['skills'].apply(ast.literal_eval)

In [29]:
print(test_tech_df.at[0, 'text'])
print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
print(test_tech_df.at[0, 'skills'])
print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
print(test_tech_df.at[0, 'knowledge'])

Full Stack Software Engineer - Java / JavaScript <ORGANIZATION> <ORGANIZATION> <ORGANIZATION> <ORGANIZATION> . <ADDRESS> <ADDRESS> <LOCATION> - <LOCATION> Date posted: 2021-03-04 Likes: 0 Dislikes: 0 Love: 0 Salary: <SALARY> Job type: FULL_TIME Experience level: <EXPERIENCE> Industry: <INDUSTRY> Company size: <SIZE> Company type: <COMPANY_TYPE> Technologies: javascript reactjs java Job description: Job type: Full-time Role: Full Stack Developer Technologies javascript reactjs java Job description As a member of our <ORGANIZATION> <ORGANIZATION> <ORGANIZATION> we look first and foremost for people who are passionate around solving business problems through innovation and engineering practices . You'll be required to apply your depth of knowledge and expertise to all aspects of the software development lifecycle as well as partner continuously with your many stakeholders on a daily basis to stay focused on common goals . We embrace a culture of experimentation and constantly strive for i

In [30]:
test_tech_df['predictions_gpt_full_text'] = None
test_tech_df['predictions_gpt_chunked'] = None

,idx,text,skills,knowledge,source,predictions_gpt_full_text,predictions_gpt_chunked
0,1,Full Stack Software Engineer - Java / JavaScri...,"[solving business problems, apply your depth o...","[javascript, reactjs, java, javascript, reactj...",tech,None,None
1,2,Software Engineer - Java <ORGANIZATION> <ORGAN...,"[solving business problems, apply your depth o...","[java, spring, java-ee, java, spring, java-ee,...",tech,None,None
2,3,DevOps Ninja Engineer ( . NET/Azure ) <ORGANIZ...,"[Open for continuous change, Enhance and maint...","[Financial Technology, SaaS, wealthtech, azure...",tech,None,None
3,4,Principle Software Engineer | Java <ORGANIZATI...,"[solving business problems, apply your depth o...","[Financial Services, java, reactjs, web-servic...",tech,None,None
4,5,Python Software Engineer Intern for <ORGANIZAT...,[build highly efficient accurate and scalable ...,"[Artificial Intelligence, Computer Graphics, S...",tech,None,None


In [4]:
test_tech_df.shape

(31, 5)

In [5]:
verbose_template = """
You are a top-notch recruiter and data labeler.
Your task is to thoroughly analyze the following part of a job posting and extract text spans including skills or knowledge.

You should use the following guideline for annotation:

Instruction for Annotating Text for Skills and Knowledge Components
Objective: Identify and annotate spans of text in job postings (JPs) that represent specific skills or knowledge required for the position.

1. Identifying Skills:

General Rule: A skill is usually indicated by a VERB or an (ADJECTIVE) + NOUN combination.
Exclusion of Modal Verbs: Do not tag modal verbs (e.g., can, will) as part of the skill.
Phrase Separation: Split phrases containing prepositions and/or conjunctions, except when conjunctions coordinate two nouns as a single argument.
Handling Anaphoric Pronouns: Avoid tagging skills with anaphoric pronouns. Only tag the preceding skill.
Splitting Coordinated Components: Split nouns and adjectives in coordination if they lack a verb.
Listing of Skills: If skills are listed leading to different subtasks, annotate each separately.
Brevity in Contextual Information: Keep skill annotations concise, especially when followed by company-specific info.
Inclusion of 'skills' or 'knowledge' in Tags: Include these words in the annotation if their omission alters the meaning.
Parenthetical Information: Include if it elaborates on the skill or is an abbreviation.
Adverbial Inclusion: Include adverbials that describe the manner of doing something.
Attitudes as Skills: Annotate attitudes as skills, omitting articles.
Exclusions:
Avoid tagging ironic skills.
Avoid nested annotations; use one span for overlapping skills.
Do not tag skills in top headlines, but do tag in sub-headlines and body text.
Exclude fluff and triggers surrounding the skill component.
Exclude participation, contributing, and transfer expressions from skill annotations.
Do not annotate occupations or positions.
Focus on skills related and specific to the position.

2. Identifying Knowledge:
General Rule: Knowledge is non-executable and possessed by an individual.
Inclusion of Parenthetical Information: Include if it is related to the knowledge component.
Licenses and Certifications: Include additional words like "certificate," "license," etc.
Vague Preceding Verbs: Only tag the knowledge component if preceded by a vague verb.
Specificity: Annotate only specified knowledge components.
Nested Knowledge in Skills: Knowledge components can be nested within skill annotations.
Coordination of Knowledge Components: Annotate as one if all components share one knowledge tag.
Listing Knowledge Tags: Annotate all knowledge tags separately.
Annotation of Industries and Fields: Tag these as knowledge components.

3. Prioritization and Other Considerations:
Prefer skills over knowledge in uncertain cases.
Prioritize skills over attitudes; only tag the skill within an attitude.
Keep annotations concise and relevant to the job.
Annotate skills and knowledge in unconventional places if related to the position.
Consider annotating a combination of skill and knowledge when applicable.
Exclude knowledge/skill components in positions.
Focus on annotations relevant to the specific position and its future expectations.

Output just the JSON array, wihtout any other text.
Start your response with "[" symbol, and finish with "]".
Make sure your response doesn't include anything else.

Job posting:
{job_posting}

Response:"""

short_template = """
    You are a top-notch recruiter and data labeler.
    Your task is to thoroughly analyze the following part of a job posting and extract text spans including skills and knowledge.
    A skill in a job context is typically indicated by a verb or an adjective-noun combination, reflecting an executable ability or a specific way of performing a task.
    Knowledge, on the other hand, refers to non-executable information that an individual possesses, often indicated by specific fields, industries, or certifications, and is distinct from direct action or skills.

    Output the JSON array, wihtout any other text.
    Start your response with "[" symbol, and finish with "]".
    Make sure your response doesn't include anything else.

    Job posting:
    {job_posting}
"""

few_shot_template = """
    You are a top-notch recruiter and data labeler.
    Your task is to thoroughly analyze the following part of a job posting and extract text spans including job-related skills or knowledge.
    A skill in a job context is typically indicated by a verb or an adjective-noun combination, reflecting an executable ability or a specific way of performing a task.
    Knowledge, on the other hand, refers to non-executable information that an individual possesses, often indicated by specific fields, industries, or certifications, and is distinct from direct action or skills.

    Below you can see some examples. Pay attention to what is included in detected spans. We want not to include false positive skills and knowledge.

    Job posting:
    Ability to work in large collaborative teams to achieve organizational goals
    Detected spans:
    ["work in large collaborative teams"]

    Job posting:
    Work hands-on together with the other engineers within the Agile team
    Detected spans:
    ["Work hands-on"]

    Job posting:
    Requirements At least 5 years of combined experience in Java or Kotlin and JavaScript or TypeScript programming and related test frameworks ( Selenium TestCafe etc.) .
    Detected spans:
    ["Java", "Kotlin", "JavaScript", "Typescript"]

    Job posting:
    A degree in Computer Science or related fields or equivalent practical experience .
    Detected spans:
    ["degree in Computer Science"]

    Job posting:
    Experience in working on a cloud-based application running on Docker .
    Detected spans:
    ["Docker", "working on a cloud-based application"]

    Make sure to never include protected attributes like gender identity or veteran status in the returned list:

    Job posting:
    We do not discriminate on the basis of any protected attribute including race religion color national origin gender sexual orientation .
    Detected spans:
    []

    Output just the JSON array, wihtout any other text.
    Start your response with "[" symbol, and finish with "]".
    Make sure your response doesn't include data from examples and anything else except from JSON array.
    Return empty array If there are no obvious skill and knowledge spans in the given posting.

    Job posting:
    {job_posting}
    Detected spans:
"""

annotation_guidelines_template = """
    You are a top-notch recruiter and data labeler.
    Your task is to thoroughly analyze the following part of a job posting and extract text spans including candidate-related skill and knowledge components.
    The definition of skill component is the following:
    "The ability to apply knowledge and use know-how to complete tasks and solve problems. Skills are described as cognitive (involving the use of logical, intuitive and creative thinking) or practical (involving manual dexterity and the use of methods, materials, tools and instruments)."
    The definition of knowledge is the following:
    "The body of facts, principles, theories and practices that is related to a field of work or study. Knowledge is described as theoretical and/or factual, and is the outcome of the assimilation of information through learning."
    Make sure to detect the shortest possible text span excluding irrelevant, company-specific information.

    Output just the JSON array, with skill or knowledge strings surrounded by quotation marks.
    Start your response with "[" symbol, and finish with "]".

    Example job posting including skill components:
    We are looking for people who want to work in cross-functional teams and want to help others learn how to build them
    Detected spans:
    ["work in cross-functional teams", "help others learn"]

    Example job posting including knowledge components:
    Experience with latest versions of Java and C++.
    Detected knowledge components:
    ["Java", "C++"]

    Make sure your response doesn't include anything else except from JSON array. Each skill should be a separate string.
    Return empty array If there are no obvious skill or knowledge components in the given posting.

    Job posting:
    {job_posting}
    Detected spans:
"""


template_collection = {
    'verbose': verbose_template,
    'short': short_template,
    'few_shot': few_shot_template,
    'annotation_guidelines_template': annotation_guidelines_template
}

In [6]:
from langchain import LLMChain, PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

prompt_template = PromptTemplate(input_variables=["job_posting"], template=template_collection['few_shot'])

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4", temperature=0)

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}")
])

In [15]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

In [39]:
import ast

def get_skills_for_posting(job_posting):
    prompt_val = prompt_template.invoke({"job_posting": job_posting})
    output = chain.invoke({"input": prompt_val.to_string()})
    array_start = output.find('[')
    array_end = output.find(']', array_start)
    array_string = output[array_start:array_end+1].strip()
    return ast.literal_eval(array_string)

gpt_results = get_skills_for_posting(test_posting)
gpt_results

['Full Stack Software Engineer',
 'Java',
 'JavaScript',
 'solving business problems through innovation and engineering practices',
 'software development lifecycle',
 'BS/BA degree',
 'Advanced knowledge of application data and infrastructure architecture disciplines',
 'Understanding of architecture and design across all systems',
 'Working proficiency in developmental toolsets',
 'Knowledge of industry-wide technology strategies and best practices',
 'work in large collaborative teams',
 'Proficiency in one or more modern programming languages - Java and Javascript',
 'Understanding of software skills such as business analysis development maintenance and software improvement',
 'In-depth HTML5 / JavaScript',
 'developing multithreaded scalable concurrent applications',
 'Node JS',
 'React']

In [40]:
def calculate_f1_score(real_values, predicted_values):
    real_values = [val.lower() for val in real_values]
    predicted_values = [val.lower() for val in predicted_values]

    tp = sum([1 for val in predicted_values if val in real_values])
    fp = sum([1 for val in predicted_values if val not in real_values])
    fn = sum([1 for val in real_values if val not in predicted_values])

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    print("Precision: " + str(precision))
    print("Recall: " + str(recall))

    return f1_score

In [41]:
merged_skills_knowledge = test_tech_df.at[0, 'skills'] + test_tech_df.at[0, 'knowledge']

In [42]:
gpt_results

['Full Stack Software Engineer',
 'Java',
 'JavaScript',
 'solving business problems through innovation and engineering practices',
 'software development lifecycle',
 'BS/BA degree',
 'Advanced knowledge of application data and infrastructure architecture disciplines',
 'Understanding of architecture and design across all systems',
 'Working proficiency in developmental toolsets',
 'Knowledge of industry-wide technology strategies and best practices',
 'work in large collaborative teams',
 'Proficiency in one or more modern programming languages - Java and Javascript',
 'Understanding of software skills such as business analysis development maintenance and software improvement',
 'In-depth HTML5 / JavaScript',
 'developing multithreaded scalable concurrent applications',
 'Node JS',
 'React']

In [43]:
calculate_f1_score(merged_skills_knowledge, gpt_results)

Precision: 0.4117647058823529
Recall: 0.2413793103448276


0.3043478260869565

In [45]:
full_text_predictions_f1_scores = []

for index, row in test_tech_df.iterrows():
    try:
        row_merged_skills_knowledge = test_tech_df.at[index, 'skills'] + test_tech_df.at[index, 'knowledge']
        res = get_skills_for_posting(row['text'])
        test_tech_df.at[index, "predictions_gpt_full_text"] = res
        f1_score = calculate_f1_score(row_merged_skills_knowledge, res)
        full_text_predictions_f1_scores.append(f1_score)
    except Exception as e:
        print(f"Failed to parse skills for row {index}: {e}")

full_text_predictions_f1_scores

Precision: 0.3333333333333333
Recall: 0.27586206896551724
Precision: 0.5
Recall: 0.20689655172413793
Precision: 0.6944444444444444
Recall: 0.423728813559322
Precision: 0.17857142857142858
Recall: 0.13513513513513514
Precision: 0.875
Recall: 0.6086956521739131
Precision: 0.4666666666666667
Recall: 0.17073170731707318
Precision: 0.5625
Recall: 0.5294117647058824
Precision: 0.65
Recall: 0.5306122448979592
Precision: 0.0625
Recall: 0.05263157894736842
Precision: 0.3181818181818182
Recall: 0.1794871794871795
Precision: 0.6222222222222222
Recall: 0.56
Precision: 0.5185185185185185
Recall: 0.3888888888888889
Precision: 0.4375
Recall: 0.21212121212121213
Precision: 0.375
Recall: 0.15789473684210525
Precision: 0.4482758620689655
Recall: 0.2708333333333333
Precision: 0.4
Recall: 0.16666666666666666
Precision: 0.7857142857142857
Recall: 0.34375
Precision: 0.4
Recall: 0.2608695652173913
Precision: 0.6666666666666666
Recall: 0.27906976744186046
Precision: 0.6896551724137931
Recall: 0.57142857142857

[0.3018867924528302,
 0.2926829268292683,
 0.5263157894736842,
 0.15384615384615385,
 0.717948717948718,
 0.25,
 0.5454545454545455,
 0.5842696629213483,
 0.05714285714285714,
 0.2295081967213115,
 0.5894736842105264,
 0.4444444444444444,
 0.2857142857142857,
 0.22222222222222218,
 0.33766233766233766,
 0.23529411764705882,
 0.4782608695652174,
 0.31578947368421056,
 0.3934426229508197,
 0.625,
 0.3947368421052631,
 0.22580645161290322,
 0.21428571428571427,
 0.5538461538461539,
 0.33333333333333326,
 0.32558139534883723,
 0,
 0.7164179104477612,
 0.3235294117647059,
 0.425531914893617,
 0.4800000000000001]

In [47]:
sum(full_text_predictions_f1_scores) / len(full_text_predictions_f1_scores)

0.37352996221064944

In [62]:
from nltk.tokenize import sent_tokenize

def divide_into_chunks(text, min_sentences=5):
    sentences = sent_tokenize(text)

    chunks = []
    current_chunk = []

    for sentence in sentences:
        current_chunk.append(sentence)
        if len(current_chunk) == min_sentences:
            chunks.append(' '.join(current_chunk))
            current_chunk = []

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

chunks = divide_into_chunks(test_posting)

In [63]:
def get_skills_for_posting_chunked(job_posting):
    chunks = divide_into_chunks(job_posting)
    responses = []

    for chunk in chunks:
        try:
            prompt_val = prompt_template.invoke({"job_posting": chunk})
            output = chain.invoke({"input": prompt_val.to_string()})
            array_start = output.find('[')
            array_end = output.find(']', array_start)

            array_string = output[array_start:array_end+1].strip()
            responses = responses + ast.literal_eval(array_string)
        except:
            print("ERROR processing chunk")

    return responses

responses = get_skills_for_posting_chunked(test_posting)

responses

['Full Stack Software Engineer',
 'Java',
 'JavaScript',
 'javascript',
 'reactjs',
 'java',
 'Full Stack Developer',
 'solving business problems through innovation and engineering practices',
 'apply your depth of knowledge and expertise to all aspects of the software development lifecycle',
 'partner continuously with your many stakeholders on a daily basis',
 'embrace a culture of experimentation',
 'constantly strive for improvement and learning',
 'work in a collaborative trusting thought-provoking environment',
 'work in a small and talented software development team',
 'BS/BA degree',
 'Advanced knowledge of application data and infrastructure architecture disciplines',
 'Understanding of architecture and design across all systems',
 'Working proficiency in developmental toolsets',
 'Knowledge of industry-wide technology strategies and best practices',
 'Ability to work in large collaborative teams',
 'Proficiency in one or more modern programming languages - Java and Javascript

In [64]:
calculate_f1_score(merged_skills_knowledge, responses)

Precision: 0.21875
Recall: 0.23333333333333334


0.22580645161290322

In [68]:
chunked_predictions_f1_scores = []

for index, row in test_tech_df.iterrows():
    try:
        row_merged_skills_knowledge = test_tech_df.at[index, 'skills'] + test_tech_df.at[index, 'knowledge']
        res = get_skills_for_posting_chunked(row['text'])
        test_tech_df.at[index, "predictions_gpt_chunked"] = res
        f1_score = calculate_f1_score(row_merged_skills_knowledge, res)
        chunked_predictions_f1_scores.append(f1_score)
    except Exception as e:
        print(f"Failed to parse skills for row {index}: {e}")

chunked_predictions_f1_scores

Precision: 0.22580645161290322
Recall: 0.23333333333333334
Precision: 0.058823529411764705
Recall: 0.03125
Precision: 0.6444444444444445
Recall: 0.47540983606557374
Precision: 0.18181818181818182
Recall: 0.3888888888888889
Precision: 0.4827586206896552
Recall: 0.6086956521739131
Precision: 0.47619047619047616
Recall: 0.5714285714285714
Precision: 0.3333333333333333
Recall: 0.5294117647058824
Precision: 0.5961538461538461
Recall: 0.6326530612244898
Precision: 0.14814814814814814
Recall: 0.42105263157894735
Precision: 0.23809523809523808
Recall: 0.2564102564102564
Precision: 0.35353535353535354
Recall: 0.6363636363636364
Precision: 0.4
Recall: 0.5882352941176471
Precision: 0.28
Recall: 0.18421052631578946
Precision: 0.125
Recall: 0.15789473684210525
Precision: 0.2631578947368421
Recall: 0.32608695652173914
Precision: 0.5882352941176471
Recall: 0.43478260869565216
Precision: 0.40476190476190477
Recall: 0.53125
Precision: 0.2727272727272727
Recall: 0.2608695652173913
Precision: 0.34375
Rec

[0.22950819672131148,
 0.04081632653061224,
 0.5471698113207547,
 0.24778761061946902,
 0.5384615384615384,
 0.5194805194805195,
 0.409090909090909,
 0.6138613861386137,
 0.2191780821917808,
 0.24691358024691357,
 0.45454545454545453,
 0.4761904761904762,
 0.2222222222222222,
 0.13953488372093023,
 0.2912621359223301,
 0.5,
 0.4594594594594595,
 0.2666666666666666,
 0.44000000000000006,
 0.634920634920635,
 0.5684210526315789,
 0.26865671641791045,
 0.22950819672131145,
 0.4565217391304348,
 0.18749999999999997,
 0.22857142857142856,
 0.038461538461538464,
 0.49411764705882355,
 0.25688073394495414,
 0.6027397260273972,
 0.5384615384615384]

In [69]:
sum(chunked_predictions_f1_scores) / len(chunked_predictions_f1_scores)

0.36667452296379077

In [75]:
test_tech_df.to_csv('./data/test_df_tech_only_with_gpt_4_predictions.csv', index=False)

In [45]:
# Check baseline performance:

In [75]:
from transformers import pipeline

token_skill_classifier = pipeline(model="jjzha/jobbert_skill_extraction", aggregation_strategy="first")
token_knowledge_classifier = pipeline(model="jjzha/jobbert_knowledge_extraction", aggregation_strategy="first")

In [79]:
def aggregate_span(results):
    new_results = []
    current_result = results[0]

    for result in results[1:]:
        if result["start"] == current_result["end"] + 1:
            current_result["word"] += " " + result["word"]
            current_result["end"] = result["end"]
        else:
            new_results.append(current_result)
            current_result = result

    new_results.append(current_result)

    return new_results

In [85]:
import spacy

nlp = spacy.load("en_core_web_sm")

def ner(text):
    output_skills = []
    output_knowledge = []

    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]

    for sentence in sentences:
        skills, knowledge = process_sentence(sentence)
        output_skills.extend(skills)
        output_knowledge.extend(knowledge)


    return {"text": text, "entities": output_skills}, {"text": text, "entities": output_knowledge}

def process_sentence(sentence):
    skills = token_skill_classifier(sentence)
    for result in skills:
        if result.get("entity_group"):
            result["entity"] = "Skill"
            del result["entity_group"]

    knowledge = token_knowledge_classifier(sentence)
    for result in knowledge:
        if result.get("entity_group"):
            result["entity"] = "Knowledge"
            del result["entity_group"]

    if len(skills) > 0:
        skills = aggregate_span(skills)
    if len(knowledge) > 0:
        knowledge = aggregate_span(knowledge)


    return skills, knowledge

In [87]:
f1_scores_baseline = []

for index, row in test_tech_df.iterrows():
    try:
        row_merged_skills_knowledge = test_tech_df.at[index, 'skills'] + test_tech_df.at[index, 'knowledge']
        ner_results = ner(row['text'])
        detected_entities = [entity for entity_group in ner_results for entity in entity_group["entities"]]
        detected_words = [entity['word'] for entity in detected_entities]

        print(row_merged_skills_knowledge)
        print(detected_words)
        f1_score = calculate_f1_score(row_merged_skills_knowledge, detected_words)
        f1_scores_baseline.append(f1_score)
    except Exception as e:
        print(f"Failed to parse skills for row {index}: {e}")

f1_scores_baseline

['solving business problems', 'apply your depth of knowledge and expertise', 'partner continuously with your many stakeholders', 'stay focused on common goals', 'motivated', 'deliver of a next-generation analytics products', 'developmental toolsets', 'work in large collaborative teams', 'achieve organizational goals', 'building an innovative culture', 'developing multithreaded scalable concurrent applications', 'javascript', 'reactjs', 'java', 'javascript', 'reactjs', 'java', 'innovation and engineering practices', 'software development lifecycle', 'BS/BA degree', 'application data and infrastructure architecture disciplines', 'architecture', 'design', 'industry-wide technology strategies and best practices', 'Java', 'Javascript', 'business analysis', 'development', 'maintenance', 'software improvement', 'HTML5', 'JavaScript', 'Node JS ( server side and tooling ) *', 'React']
['solving business problems through innovation and engineering practices', 'apply your depth of knowledge and e

['build highly efficient accurate and scalable computer vision and machine learning services', 'solve technical challenges independently', 'good communicator', 'enjoy working in a team', 'Artificial Intelligence', 'Computer Graphics', 'Software Development', 'python', 'Artificial Intelligence', 'Computer Graphics', 'Software Development', 'python', 'Computer Science', 'Python', 'English', 'PostgreSQL', 'Django', 'PostGIS', 'OpenCV', 'NumPy', 'Shapely', 'Pyhon Multithreading and Multiprocessing', 'AWS/Azure', 'Computer Vision', 'Machine Learning']
['talented', 'work in a tightly - welded and talented team', 'solve technical challenges independently', 'communicator', 'working in a team', 'Computer Graphics Software Development', 'Artificial Intelligence Computer Graphics Software Development', 'machine', 'research and development', 'degree Computer Science', 'Python', 'English', 'PostgreSQL / Django / PostGIS / OpenCV / NumPy / Shapely / Pyhon Multithreading', 'Multiprocessing AWS /', 'C

['maintain quality', 'work with AWS infrastructure', 'producing cloud native software', 'logging', 'experimentation', 'problem solver', 'curious about new technology', 'build cloud', 'takes ownership', 'learn as much as possible', 'good listener', 'able to explain concepts', 'committed to improving yourself', 'demonstrate interest in technologies that are beyond the scope of your daily workflow', 'lazy', "don't like boring repetitive tasks", 'outsource your labour to the compiler', 'applying yourself to learning', 'pragmatist', 'Financial Technology', 'c#', 'f#', 'typescript', 'Financial Technology', 'c#', 'f#', 'typescript', 'modern workflows and tooling', 'continuous integration', 'automated testing', 'domain modelling', 'ubiquitous language', 'serverless containerisation', 'dev/prod environment parity', 'C#', 'F#', 'TypeScript', 'serverless framework', 'functional programming', 'Cypress', 'Jest', 'Nunit', 'SpecFlow', 'Expecto', 'FsCheck', 'Canopy', 'functions as a service', 'hosting

['flair for gameplay', 'aesthetic know-how?', 'work closely with other programmers designers and artists', 'implementing and maintaining various platforms and systems', 'implement new game features', 'making the designs look final and pretty', 'self-driven', 'open-minded', 'not afraid to be critical', 'collaborate with your colleagues', 'find solutions', 'Implementing new gameplay features and user interfaces', 'Contributing to and maintaining existing game features', 'Participating in the development of new features and optimizing existing ones', 'Participating in the development of performant and rigid systems', 'Running tests on new code', 'making user experiences at an interactive degree', '3D game programming', 'aesthetic know-how', 'system and architectural thinking', 'developing mobile games with a large amount of content delivery', 'sense of F2P game design', 'Entertainment', 'Gaming', 'Mobile Game', 'unity3d', 'c#', 'user-experience', 'Entertainment', 'Gaming', 'Mobile Game', 

['deliver software solutions', 'creation of programs', 'configuration/customisation of platforms', 'capture & analysis of requirements', 'Requirements capture/analysis and documentation', 'Design ( and documenting ) of own solutions', 'Writing software', 'configuring software platforms', 'Troubleshooting', 'debugging code', 'Testing', 'analysis of test results', 'Documentation', 'troubleshooting IT infrastructure related issues', 'Team player', 'methodical and reliable approach', 'problem solving', 'communication skills', 'Financial Services', 'Financial Technology', 'Regulatory Reporting', 'c#', 'typescript', '. net-core', 'less', 'sql-server', 'Financial Services', 'Financial Technology', 'Regulatory Reporting', 'c#', 'typescript', '. net-core', 'less', 'sql-server', 'Microsoft', 'Web development', 'C#', '. NET Core', '. NET Framework', 'SQL', 'SQL Server', 'JavaScript', 'ASP.NET', 'Blazor', 'JavaScript', 'JQuery', 'Angular', 'React', 'Vue', 'HTML5', 'CSS ( LESS)', 'Bootstrap', 'Micr

['delivering real quality', 'Planning test activities for each build cycle', 'testing client configurations', 'Reporting the results', 'Incorporating that testing into automation', 'Reporting and documenting issues found', 'Following up on issues raised', 'communicator', 'connect together complex issues', 'appreciate quality', 'code', 'qa', 'automation', 'testing', 'qa', 'automation', 'testing', 'testing', 'automation', 'complex system']
['build out QA in our event - driven platform', 'delivering real quality in both process and product', 'Planning test activities', 'testing client configurations Executing those activities Reporting the results Incorporating that testing into automation Reporting and documenting issues found Following up on issues raised', 'communicator', 'connect together complex issues', 'quality', 'Worked in a complex system', 'automation testing', 'automation testing', 'automation', 'testing', 'automation']
Precision: 0.38461538461538464
Recall: 0.29411764705882354

['base your decisions on solid data and metrics', 'produce outstanding work', 'break challenges into bite-size tasks', 'mentor team mates', 'demand quality', 'developing a whole range of key product features', 'defining and implementing services', 'collaborate with product engineers', 'accountable for your own projects', 'trusted to deliver', 'form part of a broader team', 'takes collective responsibility', 'making success happen', 'Information Technology', 'Online Travel', 'java', 'web-services', 'python', 'distributed-system', 'amazon-web-services', 'Information Technology', 'Online Travel', 'java', 'web-services', 'python', 'distributed-system', 'amazon-web-services', 'Java', 'Python', 'Docker', 'AWS', 'Hadoop/Spark', 'SQL', 'Airflow', 'Jupyter', 'Agile/Lean', 'machine learning', 'cloud services', 'APIs']
['base your decisions on solid data and metrics', 'drive', 'produce outstanding work', 'break challenges into bite - size tasks mentor team mates', 'demand quality', 'developing a 

[0.4,
 0.35294117647058826,
 0.2905982905982906,
 0.3703703703703704,
 0.15789473684210528,
 0.3380281690140845,
 0.33333333333333326,
 0.1758241758241758,
 0.1818181818181818,
 0.23655913978494625,
 0.3191489361702128,
 0.5070422535211269,
 0.1643835616438356,
 0.43243243243243246,
 0.5161290322580645,
 0.3478260869565218,
 0.25396825396825395,
 0.20833333333333331,
 0.33333333333333337,
 0.2121212121212121,
 0.35555555555555557,
 0.38961038961038963,
 0.3636363636363636,
 0.25352112676056343,
 0.33333333333333337,
 0.47368421052631576,
 0.4067796610169492,
 0.2857142857142857,
 0.4303797468354431,
 0.36666666666666664,
 0.3218390804597701]

In [88]:
sum(f1_scores_baseline) / len(f1_scores_baseline)

0.3262195622551625